# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [39]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

In [40]:
#Assign table names to variables instead of hardcode them everywhere
song_table = 'song_table'
session_table = 'session_table'
listener_table = 'listener_table'

#### Creating list of filepaths to process original event csv data files

In [41]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    # join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [42]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [43]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

In [44]:
# Retrieve the index of each column

cols = ["artist",
"firstName of user",
"gender of user",
"item number in session",
"last name of user",
"length of the song",
"level (paid or free song)",
"location of the user",
"sessionId",
"song title",
"userId"]

for index, col in enumerate(cols):
    print(index,col)

0 artist
1 firstName of user
2 gender of user
3 item number in session
4 last name of user
5 length of the song
6 level (paid or free song)
7 location of the user
8 sessionId
9 song title
10 userId


## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [45]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [46]:
# Create a Keyspace 
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [47]:
# Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [48]:
## Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4
drop_query = "DROP TABLE IF EXISTS " + song_table
query = "CREATE TABLE IF NOT EXISTS " + song_table
query = query + """(session_id INT, 
                    item_in_session INT, 
                    artist TEXT, 
                    song TEXT, 
                    length FLOAT, 
                    PRIMARY KEY (session_id, item_in_session))"""
try:
    session.execute(drop_query)
    session.execute(query)
except Exception as e:
    print(e)
                    

The primary key in this case is session_id and item_in_session since they are used in the WHERE statement

In [49]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = 'INSERT INTO ' + song_table + """ ( session_id,
                                                    item_in_session,
                                                    artist, 
                                                    song,
                                                    length)"""
        query = query + "VALUES (%s, %s, %s, %s, %s)"
        session.execute(query, (int(line[8]),int(line[3]),line[0],line[9],float(line[5])))

#### Do a SELECT to verify that the data have been inserted into each table

In [50]:
## TO-DO: Add in the SELECT statement to verify the data was entered into the table
query = """
        SELECT 
            artist, song, length
        FROM 
        """ + song_table + \
        """
        WHERE 
            session_id = 338 AND item_in_session = 4
        """
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

template = "{0:20}|{1:40}|{2:15}"
print(template.format("artist","song","length"))
print('-'*80)
for row in rows:
    print(template.format(row.artist, row.song, row.length))

artist              |song                                    |length         
--------------------------------------------------------------------------------
Faithless           |Music Matters (Mark Knight Dub)         |495.30731201171875


### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

In [51]:
## Get name of artist, song (sorted by itemInSession)
#and user (first and last name) for userid = 10, sessionid = 182
drop_query = "DROP TABLE IF EXISTS " + session_table
query = "CREATE TABLE IF NOT EXISTS " + session_table
query = query + """(user_id INT, 
                    session_id INT, 
                    item_in_session INT,
                    artist TEXT, 
                    song TEXT, 
                    first_name TEXT, 
                    last_name TEXT, 
                    PRIMARY KEY (user_id, session_id, item_in_session))"""
try:
    session.execute(drop_query)
    session.execute(query)
except Exception as e:
    print(e)

The primary key in this case is user_id, session_id (used in WHERE statement) and item_in_session (used to sort records)

In [52]:
# Insert into the table based on the csv file

file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO " + session_table + """  (user_id,
                                                        session_id,
                                                        item_in_session,
                                                        artist, 
                                                        song,
                                                        first_name,
                                                        last_name)"""
        query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]),line[0],line[9], line[1], line[4] ))

In [53]:
## Query 2. Give me only the following: name of artist, song (sorted by itemInSession)
#and user (first and last name) for userid = 10, sessionid = 182

# Define, run and print the query
query = """
            SELECT 
                * 
            FROM 
        """ + session_table + \
        """
            WHERE 
                user_id = 10 AND session_id = 182
        """
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

template = "{0:18}|{1:53}|{2:30}"
print(template.format("artist","song","listener(full name)"))
print("-"*90)
for row in rows:
    #print(row)
    print(template.format(row.artist,row.song,row.first_name+' '+row.last_name))

artist            |song                                                 |listener(full name)           
------------------------------------------------------------------------------------------
Down To The Bone  |Keep On Keepin' On                                   |Sylvie Cruz                   
Three Drives      |Greece 2000                                          |Sylvie Cruz                   
Sebastien Tellier |Kilometer                                            |Sylvie Cruz                   
Lonnie Gordon     |Catch You Baby (Steve Pitron & Max Sanna Radio Edit) |Sylvie Cruz                   


In [54]:
## Query 3: Give every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

# create listener_table, drop if it already exists
drop_query = "DROP TABLE IF EXISTS " + listener_table
query = "CREATE TABLE IF NOT EXISTS " + listener_table
query = query + """(song TEXT,
                    user_id INT,
                    first_name TEXT, 
                    last_name TEXT, 
                    PRIMARY KEY(song, user_id))"""
try:
    session.execute(drop_query)
    session.execute(query)
except Exception as e:
    print(e)        

The primary key in this case are song (used in WHERE statement) and user_id (in order to guarantee row uniqueness).

In [55]:
# Read the csv file to insert into the table
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO " + listener_table + " (song, user_id, first_name, last_name)"
        query = query + "VALUES (%s, %s, %s, %s)"
        session.execute(query, (line[9], int(line[10]), line[1], line[4]))

In [56]:
## Query 3: Give every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'
query = """
            SELECT 
                first_name, last_name
            FROM 
        """     + listener_table + \
        """
            WHERE 
                song = 'All Hands Against His Own'
        """

#Run the query
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
#Print the result of the query
template = "{0:20}|{1:20}"
print(template.format("first_name","last_name"))
print("-"*40)
for row in rows:
    print(template.format(row.first_name, row.last_name))

first_name          |last_name           
----------------------------------------
Jacqueline          |Lynch               
Tegan               |Levine              
Sara                |Johnson             


### Drop the tables before closing out the sessions

In [57]:
try:
    session.execute("DROP TABLE IF EXISTS " + song_table)
except Exception as e:
    print(e)
    
try:
    session.execute("DROP TABLE IF EXISTS " + session_table)
except Exception as e:
    print(e)
    
try:
    session.execute("DROP TABLE IF EXISTS " + listener_table)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [58]:
session.shutdown()
cluster.shutdown()